# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not to base the power system on wind power as electricity from wind turbines was more expensive than electricity produced from coal. However, we now introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_prescaler import make_prescaler
from message_ix.tools.make_prescaler import disp_range

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

In [ ]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="emission_bound")


scen_name = "emission_bound_unscaled"
scen = base.clone(
    model,
    scen_name,
    "introducing scaling tool for westeros tutorial",
    keep_solution=False,
)
scen.check_out()

## Time to Solve the Model

In [ ]:
scen.commit(comment="Introducing scaling tool")
scen.set_as_default()

current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model}_{scen_name}.mps")

In [ ]:
scen.solve(solve_options={"writemps": mps_dir,})
scen.var("OBJ")["lvl"]

Using the unscaled scenario above, we make scaler arguments to solve scenario with scaled matrix coefficients

In [ ]:
scale_args = make_prescaler(mps_dir, scen, bounds=2, steps=1)
scale_args

## Run scenarios with scaled matrix

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="emission_bound")


sc_name = "emission_bound_scaled"
sc = base.clone(
    model,
    sc_name,
    "introducing scaling tool for westeros tutorial",
    keep_solution=False,
)
sc.check_out()

sc.commit(comment="Introducing scaling tool")
sc.set_as_default()

This Scenario has a solution, use `Scenario.remove_solution()` or `Scenario.clone(..., keep_solution=False)`


In [3]:
current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model}_{sc_name}.mps")

In [4]:
model = model
scenario_name = "emission_bound_unscaled"

sc.solve(solve_options={"writemps": mps_dir},
         #gams_args=[f"--scaler='MsgPrescaler_Westeros Electrified_emission_bound_unscaled'"]
        )

In [5]:
sc.var("OBJ")["lvl"]

206280.0625

In [6]:
lp.read_mps(mps_dir)
data = lp.read_matrix()
disp_range(data,"\nNew exp range:")


Reading MPS-format file C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_prescaler\Westeros Electrified_emission_bound_scaled.mps.
Next section found: NAME          gamsmodel (line 1).
	Problem name: gamsmodel.
Next section found: ROWS (line 2).
	Row _obj (row_seq = 0) is the objective (goal function) row.
Next section found: COLUMNS (line 74).
Next section found: RHS (line 259).
	Id of RHS: rhs
Next section found: BOUNDS (line 268).
	Id of BOUNDS: bnd
Next section found: ENDATA (line 282).

Finished processing 282 lines of the MPS file: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_prescaler\Westeros Electrified_emission_bound_scaled.mps.
LP has: 71 rows, 55 cols, 184 non-zeros, matrix density = 4.71e-02.
Numbers of redefined: RHS = 8, ranges = 0, bounds = 13.

The GF (objective) row named "_obj" has 7 elements.
Distribution of the GF (objective) values:
count    7.000000
mean     4.534986
std      2.522265
min      1.000000


## Close the connection to the database

In [7]:
mp.close_db()